<a href="https://colab.research.google.com/github/Fet-elena/proba/blob/main/HW_1_sql_Fetisova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [4]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [5]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


In [7]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [46]:
tasks+=1
pd.read_sql(
    '''
    SELECT CustomerName, Country, Address FROM Customers c
    where c.Country = 'Germany'or c.Country = 'France' or c.Country = 'Spain' and c.City = 'Madrid'
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [47]:
tasks+=1
pd.read_sql(
    '''
    SELECT c.Country, count(c.CustomerName) FROM Customers c
    group by c.Country
    ORDER BY count(c.CustomerName) DESC
    LIMIT 3
    ''',
    con,
)

,Country,count(c.CustomerName)
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [48]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM (SELECT ROW_NUMBER() over (ORDER BY date(OrderDate), s.ShipperName) as num,
    s.ShipperName, o.OrderDate
    FROM Orders o
    join Shippers s on o.ShipperID = s.ShipperID)
    WHERE num = 10
    ''',
    con,
)

,num,ShipperName,OrderDate
0,10,Federal Shipping,05.08.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [49]:
tasks+=1
pd.read_sql(
    '''
    SELECT p.ProductName, p.Unit, d.Quantity, p.Price
    FROM (SELECT o.OrderID
    FROM Orders o, OrderDetails d, Products p
    WHERE o.OrderID = d.OrderID AND d.ProductID = p.ProductID
    GROUP BY o.OrderID
    ORDER BY sum(d.Quantity*p.Price) DESC LIMIT 1) h, Orders o, OrderDetails d, Products p
    WHERE o.OrderID = h.OrderID AND d.OrderID = h.OrderID AND d.ProductID = p.ProductID
    ''',
    con,
)

,ProductName,Unit,Quantity,Price
0,Sir Rodney's Marmalade,30 gift boxes,12,81.0
1,Côte de Blaye,12 - 75 cl bottles,40,263.5
2,Camembert Pierrot,15 - 300 g rounds,70,34.0
3,Mozzarella di Giovanni,24 - 200 g pkgs.,42,34.8


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [50]:
tasks+=1
pd.read_sql(
    '''
    SELECT p.ProductName, d.OrderID, d.Quantity from(
      SELECT d.ProductID FROM OrderDetails d join Products p on d.ProductID = p.ProductID
      group by d.ProductID ORDER BY sum(d.Quantity) DESC LIMIT 1) as n
      join Products p on n.ProductID = p.ProductID
      join OrderDetails d on d.ProductID = p.ProductID
             ''',
    con,
)

,ProductName,OrderID,Quantity
0,Gorgonzola Telino,10253,20
1,Gorgonzola Telino,10272,40
2,Gorgonzola Telino,10273,15
3,Gorgonzola Telino,10325,4
4,Gorgonzola Telino,10335,25
5,Gorgonzola Telino,10342,56
6,Gorgonzola Telino,10356,30
7,Gorgonzola Telino,10359,70
8,Gorgonzola Telino,10363,20
9,Gorgonzola Telino,10374,30


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [51]:
tasks+=1
pd.read_sql(
    '''
    SELECT s.SupplierName, s.Country, s.ContactName, s.Phone FROM Suppliers s
      JOIN Products p on p.SupplierID = s.SupplierID
      JOIN OrderDetails d on p.ProductID = d.ProductID
      JOIN Orders o on o.OrderID = d.OrderID
      GROUP BY p.SupplierID ORDER BY count() DESC LIMIT 5
          ''',
    con,
)

,SupplierName,Country,ContactName,Phone
0,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
1,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
2,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
3,Norske Meierier,Norway,Beate Vileid,(0)2-953010
4,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [52]:
tasks+=1
pd.read_sql(
    '''
    SELECT C.Country, cat.CategoryName, sum(d.Quantity*Price) TotalPrice
    FROM Orders o, Customers c, OrderDetails d, Products p, Categories cat
    WHERE o.CustomerID = c.CustomerID AND c.Country = 'Brazil'
    AND o.OrderID = d.OrderID AND d.ProductID = p.ProductID AND p.CategoryID = cat.CategoryID
    GROUP BY p.CategoryID ORDER BY sum(d.Quantity*Price) DESC LIMIT 1
    ''',
    con,
)

,Country,CategoryName,TotalPrice
0,Brazil,Beverages,13690.0


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [53]:
tasks+=1
pd.read_sql(
    '''
    SELECT (max(Cena) - min(Cena)) Raznica
    FROM (SELECT o.OrderID, sum(d.Quantity*p.Price) Cena
    FROM Orders o, OrderDetails d, Products p, Customers c
    WHERE o.OrderID = d.OrderID AND p.ProductID = d.ProductID AND c.CustomerID = o.CustomerID AND c.Country = 'USA'
    GROUP BY o.OrderID) Cena
    ''',
    con,
)

,Raznica
0,7638.45


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [54]:
tasks+=1
pd.read_sql(
    '''
    SELECT count() Kol_vo, (e.FirstName || ' ' || e.LastName) FIO
    FROM Orders o, Employees e
    WHERE o.EmployeeID = e.EmployeeID
    GROUP BY e.EmployeeID
    ORDER BY e.BirthDate DESC LIMIT 3
    ''',
    con,
)

,Kol_vo,FIO
0,31,Janet Leverling
1,14,Robert King
2,40,Margaret Peacock


10. Сколько банок крабового мяса всего было заказано.

In [55]:
tasks+=1
pd.read_sql(
    '''
    SELECT sum(d.Quantity) Crab
    FROM Orders o, OrderDetails d, Products p
    WHERE o.OrderID = d.OrderID AND d.ProductID = p.ProductID AND p.ProductName = 'Boston Crab Meat'
    ''',
    con,
)

,Crab
0,256


In [64]:
if tasks>=10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
